<a href="https://colab.research.google.com/github/Faraz011/Assignment-AIML-Lab/blob/main/Dynamic_Pricing_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



reading data form source file

In [44]:
data = pd.read_csv('data.csv')
print(data.head())

         date  year  month  day_of_week  day_of_month  week_of_year  quarter  \
0  2020-01-01  2020      1            3             1             1        1   
1  2020-01-01  2020      1            3             1             1        1   
2  2020-01-01  2020      1            3             1             1        1   
3  2020-01-02  2020      1            4             2             1        1   
4  2020-01-02  2020      1            4             2             1        1   

    day_name month_name  season  ...  price_difference price_advantage  \
0  Wednesday    January  Winter  ...             -0.04               1   
1  Wednesday    January  Winter  ...              0.52               0   
2  Wednesday    January  Winter  ...              1.27               0   
3   Thursday    January  Winter  ...             -0.71               1   
4   Thursday    January  Winter  ...             -0.75               1   

   price_ratio_vs_competitor starting_inventory  ending_inventory  restock

Data preprocessing and Feature Engineering

In [32]:
numeric_feat = data.select_dtypes(include = ['int64','float64']).columns


In [34]:
skewness = data[numeric_feat].skew()
print(skewness.sort_values(ascending = False))

demand_shock                 9.331092
is_holiday                   3.454220
stockout                     2.553046
demand                       2.196976
restocked                    2.127229
is_promotion                 2.023521
revenue                      1.536622
is_weekend                   0.951068
ending_inventory             0.863663
starting_inventory           0.810312
competitor_price             0.398402
own_price                    0.309400
price_ratio_vs_competitor    0.162447
year                         0.086362
price_advantage              0.056041
quarter                      0.031676
week_of_year                 0.026061
month                        0.017617
day_of_month                 0.014588
day_of_week                  0.000917
price_difference            -0.502031
dtype: float64


In [38]:
skewed_feat = skewness[abs(skewness) > 0.1].index
print(skewed_feat)

Index(['is_weekend', 'is_holiday', 'is_promotion', 'own_price',
       'competitor_price', 'price_difference', 'price_ratio_vs_competitor',
       'starting_inventory', 'ending_inventory', 'restocked', 'stockout',
       'demand', 'demand_shock', 'revenue'],
      dtype='object')


In [39]:
data['revenue'] = np.log1p(data['revenue'])
data['price_difference'] = np.log1p(data['price_difference'])

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [41]:
print (data['price_difference'].head())
print (data['revenue'].head())

0   -0.040822
1    0.418710
2    0.819780
3   -1.237874
4   -1.386294
Name: price_difference, dtype: float64
0    7.021111
1    7.380081
2    6.257342
3    6.911847
4    6.945764
Name: revenue, dtype: float64


In [46]:
data['price_x_promotion'] = data['own_price'] * data['is_promotion']
data['price_x_weekend'] = data['own_price'] * data['is_weekend']


print(data.head())

         date  year  month  day_of_week  day_of_month  week_of_year  quarter  \
0  2020-01-01  2020      1            3             1             1        1   
1  2020-01-01  2020      1            3             1             1        1   
2  2020-01-01  2020      1            3             1             1        1   
3  2020-01-02  2020      1            4             2             1        1   
4  2020-01-02  2020      1            4             2             1        1   

    day_name month_name  season  ...  price_ratio_vs_competitor  \
0  Wednesday    January  Winter  ...                   0.998822   
1  Wednesday    January  Winter  ...                   1.013815   
2  Wednesday    January  Winter  ...                   1.070477   
3   Thursday    January  Winter  ...                   0.975826   
4   Thursday    January  Winter  ...                   0.984349   

  starting_inventory  ending_inventory restocked  stockout  demand  \
0                500               467        

In [22]:
split_indx = int(len(data) * 0.8)
train_data = data[:split_indx]
test_data = data[split_indx:]

X_train = train_data[[
    'own_price', 'competitor_price', 'is_promotion', 'price_difference',
    'price_x_promotion', 'is_weekend', 'day_of_week', 'month', 'demand'
]]
y_train = train_data['revenue']
X_test = test_data[[
    'own_price', 'competitor_price', 'is_promotion', 'price_difference',
    'price_x_promotion', 'is_weekend', 'day_of_week', 'month', 'demand'
]]
y_test = test_data['revenue']

In [26]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import PoissonRegressor

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = PoissonRegressor(alpha = 0.1,max_iter = 1000)
model.fit(X_train_scaled, y_train)

PoissonRegressor(alpha=0.1, max_iter=1000)

In [33]:
from sklearn.metrics import accuracy_score
predict = model.predict(X_test_scaled)


In [54]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predict)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 449609.06927813916


In [52]:
mean = data['revenue'].mean()
print(f'Mean Revenue: {mean}')

Mean Revenue: 1626.0099


Implementing XGBoost

In [50]:
from xgboost import XGBRegressor, XGBClassifier

xgb_model = XGBRegressor(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42
)

xgb_model.fit(X_train, y_train)  # No scaling needed

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=None, num_parallel_tree=None, ...)

In [49]:
!pip install xgboost

In [56]:
predict1 = xgb_model.predict(X_test)

In [57]:
mse1 = mean_squared_error(y_test, predict1)
print(f'Mean Squared Error: {mse1}')

Mean Squared Error: 11060.424609051051


In [58]:
rmse = np.sqrt(mse1)
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 105.16855332774647
